In [2]:
# calculating raw photon counts should produce same results as saveed by SM
import import_ipynb
from rescale import get_rescaled_wave_soln
import os
import numpy as np
import pandas as pd
import astropy.io.fits 
wave_soln_ref = np.array(get_rescaled_wave_soln())

Finished rescale


In [30]:
# define new resampling function for baseline photons -> sum not average in overlap regions
from rescale import resample_order
def new_resample_baseline(wave_soln_ref, wave_soln, arr):
    resampled_orders = np.empty(shape=(0)) 
    resampled_wl = np.empty(shape=(0)) # contains resampled wl for each order concatenated (so there are repeats)
    for order in np.arange(30,52,1):
        #order_values = arr[order][100:-101] # truncate the ends becuase the deblazing doesn't work well here --> unless in future iteration do this in deblazing
        #apf_wl_values = wave_soln[order][100:-100]
        order_values = arr[order][:-1] 
        apf_wl_values = wave_soln[order][:-1]
        first = apf_wl_values[0]
        last = apf_wl_values[-1]
        new_first = first - 0.017 
        new_last = last + 0.17 
        new_wl_section = np.array(wave_soln_ref)[(new_first <= wave_soln_ref) * (new_last >= wave_soln_ref)]
        resampled_order = resample_order(new_wl_section, apf_wl_values, order_values)
        resampled_orders = np.hstack([resampled_orders, resampled_order])
        resampled_wl = np.hstack([resampled_wl, new_wl_section])
    # sum duplicate flux values
    resampled_arr = np.zeros(len(wave_soln_ref))
    i=0
    for wl in wave_soln_ref:
        values = resampled_orders[resampled_wl == wl]
        total = np.nansum(values)
        resampled_arr[i] = total
        i += 1       
    return resampled_arr

In [31]:
test = '3'
#star_list = os.listdir('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra_' + test)
star_list = os.listdir('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra_' + test)
try: star_list.remove('.ipynb_checkpoints')
except: pass
photon_counts_path = 'SM_photon_counts_all_injections/test_' + test
if not os.path.isdir(photon_counts_path):
    os.mkdir(photon_counts_path)
apf_wl_soln = (astropy.io.fits.open('apf_wav.fits'))[0].data

for star in star_list:
    
    star_name = star.split('_')[0]
    
    fits_list = os.listdir('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra_' + test + '/' + star)
    try: fits_list.remove('.ipynb_checkpoints')
    except: pass
    reduced_spect = np.zeros((79, 4608))
    for i in range(len(fits_list)):
        fits_file = fits_list[i]
        if not '_injection' in fits_file: # this would be a mistake file leftoever from a previous test. 
            hdu = astropy.io.fits.open('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra_' + test + '/' + star + '/' + fits_file)
            data_part = hdu[0].data
            reduced_spect += data_part

    resamp_no_deblaze = new_resample_baseline(wave_soln_ref, apf_wl_soln, reduced_spect)
    photon_counts_path = 'SM_photon_counts_all_injections/test_' + test
    photons_file = photon_counts_path + '/photon_counts_' + star_name + '.csv'
    
    if not os.path.exists(photons_file):
        print(photons_file)
        photons_df = pd.DataFrame(resamp_no_deblaze)
        photons_df.to_csv(photons_file, index = False)

SM_photon_counts_all_injections/test_3/photon_counts_HIP13834.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ1002.csv
SM_photon_counts_all_injections/test_3/photon_counts_HIP115562.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ144.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ234.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ244.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ280.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ54.1.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ628.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ65A.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ687.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ702B.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ768.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ820A.csv
SM_photon_counts_all_injections/test_3/photon_counts_GJ83.1.csv
SM_photon_counts_all_injections/test_3/phot